In [1]:
import torch
import matplotlib.pyplot as plt
from dataloader import MRIDataset
# from residual3dunet.model import ResidualUNet3D
from residual3dunet.res3dunetmodel import ResidualUNet3D
from torch.utils.data import Dataset, DataLoader
from utils import compute_average, add_mask_colour, greytoRGB, plot_overlapped, preprocess, read_data_as_numpy, write_to_h5, read_data_from_h5
import torchvision.transforms as T
import nibabel as nib
import numpy as np

In [9]:
# write_to_h5('./dataset/test/T2', 'T2test')



# x = read_data_from_h5('./dataset/T2train.h5', 2)
# y = read_data_from_h5('./dataset/T2trainmask.h5', 2)
# print(x.shape)

# print(torch.amax(x))


# for i in range(14):
#     fig = plt.figure()
#     plt.imshow(x[0,i,:,:], cmap='gray')
#     plt.imshow(y[0,i,:,:], cmap='gray',alpha=0.3)
#     plt.axis('off')
#     plt.show()

In [ ]:
plotloss('output/train16.out')
plt.figure()
# plotaccuracy('output/train6.out')


In [ ]:
device = torch.device('cpu')
model = ResidualUNet3D(in_channels=1, out_channels=1, testing=True).to(device)

# model = torch.nn.DataParallel(model)
model.load_state_dict(torch.load("model28.pt", map_location=device))

In [ ]:
# testdataset = MRIDataset(train=True, transform=T.Compose([T.ToTensor()]))
# test_loader = DataLoader(dataset = testdataset, batch_size=1, shuffle=False)

# print(len(test_loader))


for i in range(1,50):
    image_path = f'./dataset/test/T2/MRI{i}_T2.nii.gz'
    image_data = preprocess(read_data_as_numpy(image_path))

    # image_data = preprocess(image_data)
    print(image_data.shape)
    # print(torch.amax(image_data))


# # Plot 3D data depth-wise
# for i in range(14):
#     fig = plt.figure()
#     plt.imshow(image_data[i,:,:], cmap='gray')
#     plt.axis('off')
#     plt.show()


In [ ]:
# model.eval()

# for data, target in test_loader:
#     print(data.shape)
    # print(target.shape)

# dataiter = iter(test_loader)
# data = dataiter.next()
# # data = dataiter.next()
# # data = dataiter.next()
# features, labels = data

# features, labels = features.float().to(device), labels.float().to(device)

# # image_data = image_data.float().to(device)
# print(torch.amax(features))
# # print(features)
# # output = model(features)

In [ ]:

for i in range(14):
    fig = plt.figure()
    plt.imshow(x[0,i,:,:], cmap='gray')
    plt.axis('off')
    plt.show()

In [ ]:
# preds = torch.sigmoid(output)
preds = (output > 0.5).float()

# print(preds.shape)
# print(labels.shape)
batch, channel, depth, width, height = preds.shape


print(features.shape)
print(preds.shape)
print(labels.shape)
# print(dice_coefficient(preds, labels))


In [ ]:
from utils import to_depth_last, convert_to_numpy
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches

def plot_overlapped(feature, prediction, target, size=(3,3)):
    """Plot the feature, predicted mask and groundtruth overlapping each other.
    Args:
        feature (np.ndarray/torch.tensors): A 3D or a 4D array/tensor, the original feature image
        prediction (np.ndarray/torch.tensors): A 3D or a 4D array/tensor, the predicted mask
        target (np.ndarray/torch.tensors): A 3D or a 4D array/tensor, the ground truth mask
    """
    # Plotting the data, prediction and target, overlapping each other

    assert len(feature.shape) in (3,4), 'feature must contain 3 or 4 dimensions: (W*H*D) or (C*W*H*D)'
    assert len(prediction.shape) in (3,4), 'prediction must contain 3 or 4 dimensions: (W*H*D) or (C*W*H*D)'
    assert len(target.shape) in (3,4), 'target must contain 3 or 4 dimensions: (W*H*D) or (C*W*H*D)'

    # Preprocessing feature, prediction and target
    # feature = convert_to_numpy(to_depth_last(feature))
    feature = convert_to_numpy(to_depth_last(feature))
    prediction = convert_to_numpy(to_depth_last(prediction))
    target = convert_to_numpy(to_depth_last(target))

    depth = prediction.shape[-1]

    # Convert to 3D
    feature = np.squeeze(feature)
    prediction = np.squeeze(prediction)
    target = np.squeeze(target)

    prediction = add_mask_colour(prediction, "red")
    target = add_mask_colour(target, "blue")

    overlap = prediction + target

    # feature[overlap.astype(bool)] = 1

    # Colour labelling
    colors = [(1,0,0,1), (0,0,1,1), (1,0,1,1)]
    values = ['prediction', 'target', 'overlapped']
    patches = [ mpatches.Patch(color=colors[i], label=values[i] ) for i in range(len(values)) ]

    # Save image to pdf
    pdf = PdfPages("result1.pdf")

    # Plot 3D data depth-wise
    for i in range(depth):
        fig = plt.figure()
        plt.imshow(feature[:,:,i], cmap='gray')
        plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0. )
        plt.axis('off')
        plt.show()

        pdf.savefig(fig)

    pdf.close()


In [ ]:
out = plot_overlapped(features[0,:,:,:,:], preds[0,:,:,:,:], labels[0,:,:,:,:])

In [ ]:
from ipywidgets import interact
import matplotlib.patches as mpatches

# Colour labelling
colors = [(1,0,0,1), (0,0,1,1), (1,0,1,1)]
values = ['prediction', 'target', 'overlapped']
patches = [ mpatches.Patch(color=colors[i], label=values[i] ) for i in range(len(values)) ]

def explore_3d_image(layer):
    
    plt.figure(figsize=(5,10))
    plt.imshow(out[:,:,layer,:],cmap='gray')
    plt.title('Explore Segmented MRI')
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0. )
    plt.axis('off')
    return layer

interact(explore_3d_image, layer=(0,out.shape[2]-1))

In [ ]:
# Visualize Single Image Data
f, axarr = plt.subplots(14,3,figsize=(50,50))

for i in range(depth):
    axarr[i,0].imshow(features[0,0,i,:,:],cmap='gray')
    axarr[i,1].imshow(preds[0,0,i,:,:],cmap='gray')
    axarr[i,2].imshow(labels[0,0,i,:,:],cmap='gray')
    plt.axis('off')

In [ ]:
from segmentation_statistics import SegmentationStatistics

preds = preds.numpy()
labels = labels.numpy()
prediction = preds.astype(bool)
gt = labels.astype(bool)

# preds = preds.bool()
# labels = labels.bool()

stat = SegmentationStatistics(prediction[0,0,:,:,:], gt[0,0,:,:,:], (3,2,1))

In [ ]:
stat.print_table()

In [ ]:
labels = add_mask_colour(labels, "blue")
preds = add_mask_colour(preds, "red")
overlap = labels + preds

features =torch.squeeze(features, 0)
overlapped = overlap.astype(bool)
testfeatures = features

In [ ]:


#     plt.figure(figsize=(5,5))
#     plt.imshow(features[0,0,layer,:,:],cmap='gray')
#     plt.imshow(overlap[layer,:,:,:], alpha=0.8)
#     plt.title('Explore Segmented MRI')
#     plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0. )
#     plt.axis('off')
#     # return layer

# interact(explore_3d_image,layer=(0,features.shape[2]-1))

In [ ]:
for i in range(depth):
    plt.imshow(overlap[i,:,:,:])
    plt.show()